# Get Started
## Inicializando o LLM
Testando com o LLM local, Ollama. É necessário ter o Ollama rodando com o modelo llama2


In [2]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

## Invocando o LLM 

In [3]:
llm.invoke("how can langsmith help with testing?")

'\nLangsmith is a tool that can help with testing by providing a way to generate test cases and execute them automatically. Here are some ways in which Langsmith can help with testing:\n\n1. Automated Testing: Langsmith can be used to automate the testing process, reducing the time and effort required to write and run tests. It can generate test cases based on the code structure and functionality, allowing developers to focus on other tasks.\n2. Code Coverage Analysis: Langsmith can analyze the test coverage of a codebase, identifying areas that are not well-tested or have low test coverage. This helps developers identify areas that need more testing attention.\n3. Test Prioritization: Langsmith can prioritize tests based on risk and complexity, helping developers focus their testing efforts on the most critical areas of the codebase.\n4. Test Data Generation: Langsmith can generate test data automatically, reducing the time and effort required to create test data manually. This can be

## Adicionando um prompt

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

## Combinando o prompt no LLM 

In [5]:
chain = prompt | llm 

## Invocando o modelo através da chain

In [6]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is a powerful tool that can greatly benefit the testing process in several ways. Here are some of the ways Langsmith can help with testing:\n\n1. Automated Testing: Langsmith's AI-powered language model can be used to automate the testing process by generating test cases based on the documentation. This can save time and resources, as well as reduce the likelihood of human error.\n2. Test Case Generation: Langsmith can generate test cases based on the documentation, which can help ensure that all possible scenarios are covered. This can improve test coverage and reduce the risk of missing important test cases.\n3. Defect Detection: Langsmith's AI-powered language model can analyze the documentation and identify potential defects or issues. This can help developers fix problems early in the development cycle, reducing the likelihood of critical errors making it to production.\n4. Test Data Generation: Langsmit

## Incluindo um parser na chain

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is a versatile tool that can greatly assist in the testing process. Here are some ways Langsmith can help:\n\n1. Automated Testing: Langsmith's natural language processing (NLP) capabilities allow it to generate test cases based on the documentation it has been trained on. This means that Langsmith can automatically create test cases for functionalities described in the documentation, saving time and effort compared to manual testing.\n2. Test Data Generation: Langsmith can also generate test data based on the documentation, which can be used to test the system's functionality. For example, if a technical document describes a feature that takes a user input, Langsmith can generate test cases with different inputs to verify its functionality.\n3. Test Case Prioritization: With Langsmith's ability to analyze and understand the documentation, it can prioritize test cases based on their importance and risk level.

## Fazendo o download do conteúdo para o contexto

In [8]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

## Inicializando o Embeddings 

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

## Construindo os índices com o Faiss

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

## Remontando a chain para utilizar o contexto

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

## Criando um Retriver

In [12]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

## Invocando o modelo com o retrieval

In [14]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


Based on the provided context, LangSmith can help with testing by providing a variety of features that support the development, monitoring, and evaluation of LLM applications. Here are some ways LangSmith can assist with testing:

1. Prototyping: LangSmith allows users to quickly experiment with different prompts, model types, and retrieval strategies, enabling them to understand how their LLM application is performing and debug where it is failing.
2. Debugging: When things go wrong (e.g., unexpected end results, infinite agent loops, slower than expected execution, higher than expected token usage), LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues.
3. Testing: LangSmith enables users to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. Users can also run custom evaluations (both LLM and heuristic-based) to score 

## Ajustando o retrieaval para acumular o histórico

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing‚ÄãBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it‚Äôs important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it‚Äôs breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback‚ÄãWhen launching your application to an initial set of users, it‚Äôs important to gather human feedback on the responses it‚Äôs producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftenti